In [33]:
import numpy as np
import pandas as pd
import random
import warnings

from scipy import sparse, linalg, stats
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from time import time

    Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:

    MovieLens — источник данных.

    Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.

    Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

    А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать. 

    Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

    С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм

    Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

    Среди сгенерированных пар могут быть и дубликаты, удалите их.

    Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

    Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

    Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.



In [17]:
warnings.filterwarnings('ignore')

In [62]:
RANDOM_STATE = 40

In [19]:
data = pd.read_csv('ratings_df_sample_2.csv')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040099 entries, 0 to 6040098
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 184.3 MB


In [21]:
data.isna().sum() # сумма пустых ячеек

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

всего 6_040_099 строк с данными в 4 столбцах среди которых нет пустых данных
для дальйнейшей работы необходимо сгенерировать новые фичи

In [22]:
data['watched']=1

In [23]:
data.sample(10)

,userId,movieId,rating,timestamp,watched
3490113,39579,4973,5.00,1038012572,1
1861767,77385,7254,4.50,1097076781,1
5666388,35526,919,5.00,1015027681,1
1967718,81641,3753,3.00,1281901270,1
3604981,66362,800,4.00,1314423907,1
23040,982,4855,4.00,1141549147,1
1654848,69064,1275,4.00,1375067919,1
5391502,53601,30749,4.00,1226696078,1
4544032,64615,74458,4.00,1381524203,1
5686369,51312,1635,4.00,1166627242,1


In [24]:
%%time
movies_value = data['movieId'].unique()
data['movieId'] = data['movieId'].apply(lambda f:np.where(movies_value == f)[0][0])

CPU times: total: 36.3 s
Wall time: 36.9 s


In [26]:
%%time
users_values = data['userId'].unique()
data['userId'] = data['userId'].apply(lambda f:np.where(users_values == f)[0][0])

CPU times: total: 1min 31s
Wall time: 1min 33s


In [27]:
data.head()

,userId,movieId,rating,timestamp,watched
0,0,0,3.00,974918176,1
1,0,1,5.00,974836809,1
2,0,2,4.00,974837760,1
3,0,3,4.00,974837760,1
4,0,4,5.00,974840217,1


In [28]:
data.describe()

,userId,movieId,rating,timestamp,watched
count,6040099.00,6040099.00,6040099.00,6040099.00,6040099.00
mean,9127.36,422.90,3.55,1115774334.98,1.00
std,5573.44,283.85,1.00,135843321.30,0.00
min,0.00,0.00,0.50,824835410.00,1.00
25%,4365.00,165.00,3.00,995660158.00,1.00
50%,8679.00,415.00,4.00,1111706240.00,1.00
75%,13796.00,646.00,4.00,1213151458.50,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00


In [29]:
users = data.userId.unique()
movies = data.movieId.unique()

In [30]:
%%time
random_pairs = []
for item in range(6040099*2):
    pair = (random.choice(users), random.choice(movies))
    random_pairs.append(pair)

CPU times: total: 11.6 s
Wall time: 11.7 s


In [39]:
%%time
random_pairs = pd.DataFrame(random_pairs)
random_pairs.columns = ['userId', 'movieId']
random_pairs.head()

CPU times: total: 1min 19s
Wall time: 1min 20s


,userId,movieId
0,14290,379
1,18210,597
2,1742,173
3,4850,889
4,2800,599


In [40]:
random_pairs.shape

(12080198, 2)

In [41]:
random_pairs.duplicated().sum() # поиск дупликатов для дальнейшей очистки массива данных от дупликатов

3012445

In [44]:
random_pairs.drop_duplicates(inplace=True)

In [47]:
random_pairs.shape

(9067753, 2)

Нужно оставить среди созданных пар те, в которых фильм не был просмотрен

In [48]:
originals_pairs = data[['userId','movieId']].copy()

In [49]:
%%time
random_pairs = set(tuple(x) for x in random_pairs.values)
originals_pairs = set(tuple(x) for x in originals_pairs.values)

CPU times: total: 15.9 s
Wall time: 16.2 s


In [50]:
%%time
random_pairs_not_watched = pd.DataFrame(random_pairs.difference(originals_pairs))

CPU times: total: 47.3 s
Wall time: 47.7 s


In [51]:
random_pairs_not_watched.head()

,0,1
0,12885,895
1,15557,614
2,2236,469
3,4908,188
4,10059,709


In [52]:
random_pairs_not_watched.shape

(6327975, 2)

Сгенерерованных данных больше, чем исходных. Для корректной работы нужно убрать лишнее.

In [53]:
random_pairs_not_watched = random_pairs_not_watched.sample(6040099)
random_pairs_not_watched.columns = ['userId', 'movieId']

In [55]:
data = pd.concat([data, random_pairs_not_watched], ignore_index=True)

In [58]:
data.sample(10)

,userId,movieId,rating,timestamp,watched
800121,2324,794,3.00,1243911356.00,1.00
10280081,16929,911,NaN,NaN,NaN
5779937,18654,984,4.00,1252185648.00,1.00
375842,1094,128,5.00,1092101506.00,1.00
4392006,13291,118,3.00,992992834.00,1.00
2110902,6073,984,5.00,1169403802.00,1.00
10546604,4588,375,NaN,NaN,NaN
9557900,16569,105,NaN,NaN,NaN
3237306,9315,313,2.50,1197174147.00,1.00
7544219,16934,599,NaN,NaN,NaN


Данные без каких-либо значений должны быть равны нулю

In [59]:
data = data.fillna(0)

In [60]:
data.sample(10)

,userId,movieId,rating,timestamp,watched
858530,2479,745,4.50,1072756381.00,1.00
1000166,2900,622,3.50,1134736944.00,1.00
123365,363,334,5.00,995000055.00,1.00
148505,442,44,3.50,1160417244.00,1.00
5112234,15904,11,4.00,945698524.00,1.00
8735348,14414,786,0.00,0.00,0.00
6873807,3421,681,0.00,0.00,0.00
3795284,11147,292,4.00,942860477.00,1.00
7987151,2287,296,0.00,0.00,0.00
10213505,5461,899,0.00,0.00,0.00


после подготовки данных нужно заготовить тестовые и тренировочные данные

In [63]:
train, test = train_test_split(data, test_size=0.1, random_state=RANDOM_STATE)

In [64]:
features_train = train.drop('watched', axis=1)
target_train = train['watched']
features_test = test.drop('watched', axis=1)
target_test = test['watched']

In [66]:
features_train.shape, target_train.shape

((10872178, 4), (10872178,))

In [67]:
features_test.shape, target_test.shape

((1208020, 4), (1208020,))

In [68]:
def dummy_model(features):
    return [random.random() for item in range(len(features))]

In [69]:
dummy_prediction = dummy_model(features_test)
roc_auc_dummy = round((roc_auc_score(target_test, dummy_prediction)), 4)

In [78]:
data_res = {'ROC-AUC':[roc_auc_dummy]}
data_res = pd.DataFrame(data_res)
data_res = data_res.rename(index={0:'dummy_model'})
data_res.style.highlight_max(axis=0, color='#5C61')

,ROC-AUC
dummy_model,0.500400


Коллаборативная фильтрация

In [80]:
n_users = train['userId'].nunique()
n_users

20000

In [81]:
n_movies = train['movieId'].nunique()
n_movies

1000

In [82]:
%%time
train_data_matrix = np.array(pd.pivot_table(train, values='watched', index='userId', columns='movieId', fill_value=0))

CPU times: total: 9.47 s
Wall time: 9.58 s


In [83]:
train_data_matrix.shape

(20000, 1000)

In [84]:
%%time
user_similarity = cosine_distances(train_data_matrix)

CPU times: total: 12.2 s
Wall time: 12.4 s


In [85]:
user_similarity

array([[0.        , 0.52233421, 0.48253634, ..., 0.88420411, 0.99552428,
        0.97157132],
       [0.52233421, 0.        , 0.44938462, ..., 0.75245098, 0.78107931,
        0.91086339],
       [0.48253634, 0.44938462, 0.        , ..., 0.81070059, 0.76257113,
        0.89775656],
       ...,
       [0.88420411, 0.75245098, 0.81070059, ..., 0.        , 0.6833843 ,
        0.75101085],
       [0.99552428, 0.78107931, 0.76257113, ..., 0.6833843 , 0.        ,
        0.8045613 ],
       [0.97157132, 0.91086339, 0.89775656, ..., 0.75101085, 0.8045613 ,
        0.        ]])

In [86]:
%%time
movie_similarity = cosine_distances(train_data_matrix.T)

CPU times: total: 594 ms
Wall time: 625 ms


In [87]:
movie_similarity

array([[0.        , 0.45003848, 0.43902138, ..., 0.77285966, 0.63280724,
        0.80401796],
       [0.45003848, 0.        , 0.28300816, ..., 0.69395282, 0.57535057,
        0.70921377],
       [0.43902138, 0.28300816, 0.        , ..., 0.70084698, 0.53799566,
        0.7468385 ],
       ...,
       [0.77285966, 0.69395282, 0.70084698, ..., 0.        , 0.79696842,
        0.67823177],
       [0.63280724, 0.57535057, 0.53799566, ..., 0.79696842, 0.        ,
        0.85775494],
       [0.80401796, 0.70921377, 0.7468385 , ..., 0.67823177, 0.85775494,
        0.        ]])

In [88]:
user_similarity.shape

(20000, 20000)

In [89]:
movie_similarity.shape

(1000, 1000)

In [90]:
%%time
top = 10
top_similar_users = []
for i in range(n_users):
    neighbors = (user_similarity[i]).argsort()[1:top+1]
    top_similar_users.append(
        train_data_matrix[neighbors]
    )
top_similar_users = np.array(top_similar_users)

CPU times: total: 34.8 s
Wall time: 35.4 s


In [91]:
top_similar_users.shape

(20000, 10, 1000)

Матрица предсказаний 

In [92]:
predicted_movies_pop_user_based = top_similar_users.mean(1)

In [93]:
predicted_movies_pop_user_based.shape

(20000, 1000)

создание предикта для тестового датасета

In [94]:
def round_to_nearest_0_5(n):
    return round(n*2)/2

In [95]:
test['predict_user_based'] = test.apply(
    lambda f: round_to_nearest_0_5(predicted_movies_pop_user_based[int(f['userId']), int(f['movieId'])]), axis=1
)

In [97]:
test.head()

,userId,movieId,rating,timestamp,watched,predict_user_based
6792316,1905,528,0.00,0.00,0.00,0.50
4486906,13634,282,4.00,1299619302.00,1.00,1.00
151051,449,337,4.00,1296174982.00,1.00,0.50
5588817,17822,459,4.00,1360109732.00,1.00,1.00
10719956,7438,464,0.00,0.00,0.00,0.00


In [98]:
prediction = np.where(test['predict_user_based']>= 0.5, 1, 0)
roc_auc_user_based = round(roc_auc_score(prediction, test['watched']), 4)

In [101]:
data_res.loc[len(data_res.index)] = [roc_auc_user_based]
data_res = data_res.rename(index={1: 'user_based'})
data_res.style.highlight_max(axis=0, color='#5C61')

,ROC-AUC
dummy_model,0.500400
user_based,0.779100
2,0.779100
3,0.779100


ROC-AUC лучше, чем на dummy_model

In [102]:
top = 10
top_similar_movies = []
for i in range(n_movies):
    neighbors = (movie_similarity[i]).argsort()[1:top + 1]
    top_similar_movies.append( train_data_matrix.T[neighbors])
top_similar_movies = np.array(top_similar_movies)

In [104]:
top_similar_movies.shape

(1000, 10, 20000)

Необходимо просчитать среднее значение популярности для пользователя

In [105]:
predicted_movies_watched_item_based = top_similar_movies.mean(1).T
predicted_movies_watched_item_based.shape

(20000, 1000)

Предсказание для тестового датасета

In [109]:
test['predict_item_based'] = test.apply(lambda f: round_to_nearest_0_5(predicted_movies_watched_item_based[int(f['userId']),\
                                                                                                          int(f['movieId'])]), axis=1)

In [110]:
test.head()

,userId,movieId,rating,timestamp,watched,predict_user_based,predict_item_based
6792316,1905,528,0.00,0.00,0.00,0.50,0.50
4486906,13634,282,4.00,1299619302.00,1.00,1.00,1.00
151051,449,337,4.00,1296174982.00,1.00,0.50,1.00
5588817,17822,459,4.00,1360109732.00,1.00,1.00,1.00
10719956,7438,464,0.00,0.00,0.00,0.00,0.00


In [111]:
prediction = np.where(test['predict_item_based']>= 0.5 , 1 , 0)
roc_auc_item_based = round(roc_auc_score(prediction, test['watched']), 4)

In [113]:
data_res.loc[len(data_res.index)] = [roc_auc_item_based]
data_res = data_res.rename(index={2: 'item_based'})
data_res.style.highlight_max(axis=0, color='#5C61')

,ROC-AUC
dummy_model,0.500400
user_based,0.779100
item_based,0.779100
3,0.779100
4,0.768800
5,0.768800


ITEM_BASED подход немного хуже чем user_based (0.768 и 0.779 соответственно), но все равно лучше dummy_model

SVD

In [114]:
train_data_matrix = train_data_matrix.astype(float)

In [115]:
%%time
u, s, vh = svds(train_data_matrix, k=50)
s_diag_matrix = np.diag(s)

users = np.dot(u, s_diag_matrix)
items = vh.T 

CPU times: total: 9.91 s
Wall time: 2.39 s


In [116]:
np.diag(s).shape, users.shape, items.shape

((50, 50), (20000, 50), (1000, 50))

In [117]:
test['svd_prediction'] = test.apply(
    lambda f: round_to_nearest_0_5(np.dot(users[int(f['userId'])], items[int(f['movieId'])])), axis=1
)

In [118]:
test.head()

,userId,movieId,rating,timestamp,watched,predict_user_based,predict_item_based,svd_prediction
6792316,1905,528,0.00,0.00,0.00,0.50,0.50,0.50
4486906,13634,282,4.00,1299619302.00,1.00,1.00,1.00,1.00
151051,449,337,4.00,1296174982.00,1.00,0.50,1.00,0.50
5588817,17822,459,4.00,1360109732.00,1.00,1.00,1.00,0.50
10719956,7438,464,0.00,0.00,0.00,0.00,0.00,0.00


In [120]:
prediction = np.where(test['svd_prediction']>= 0.5, 1, 0)
roc_auc_svd = round(roc_auc_score(prediction, test['watched']), 4)

In [121]:
data_res.loc[len(data_res.index)] = [roc_auc_svd]
data_res = data_res.rename(index={4: 'svd_prediction'})
data_res.style.highlight_max(axis=0, color='#5C61')

,ROC-AUC
dummy_model,0.500400
user_based,0.779100
item_based,0.779100
3,0.779100
svd_prediction,0.768800
5,0.768800
6,0.813400


было реализовано три алгоритма коллаборативной фильтрации: user,item-based и алгоритм на основе матричной факторизации. Наилучшее значение принимает SVD => этот алгоритм справился лучше всего с отработкой рекомендаций